In [2]:
import pandas as pd
import numpy as np
import scipy.optimize as optimize
import os
import matplotlib.pyplot as plt
import torch

In [3]:
%matplotlib widget
plt.rcParams["figure.autolayout"] = True
plt.rcParams["figure.figsize"] = [7,5]

# Load Data

In [6]:
inputs = pd.read_csv('../Data/all_inputs_cleaned.csv')
inputs.drop(['Unnamed: 0'], axis = 1, inplace = True)
inputs

,Month,CPI,InterestRate,GDP,Borrowing,CommercialLoan,ConsumerLoan,Deficit,ITBPrice,ITBVol,VGTPrice,VGTVol,S&P500Price,S&P500Vol
0,2006-05-01,0.15045,4.35730,0.687860,0.25008,11.5,4.1,-0.284777,NaN,NaN,-0.049741,0.757681,NaN,NaN
1,2006-06-01,0.50075,3.13152,0.687860,-0.40885,17.8,5.4,-2.393523,-0.141342,0.058676,-0.058480,-0.357298,-0.141342,0.058676
2,2006-07-01,0.29895,1.01215,0.687860,1.64957,20.1,7.8,-1.361045,-0.087839,2.328675,-0.036031,0.028369,-0.087839,2.328675
3,2006-08-01,0.24839,5.01002,0.378120,0.75333,22.3,4.5,-1.478174,-0.021042,0.796079,0.032067,-0.368216,-0.021042,0.796079
4,2006-09-01,0.54509,0.19084,0.378120,-0.89659,17.7,5.6,-2.616416,0.046412,2.381466,0.066667,0.267074,0.046412,2.381466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2023-07-01,0.12413,0.39526,0.530580,0.16145,-2.6,6.0,-2.364214,0.079389,0.066784,0.040382,-0.028132,0.079389,0.066784
203,2023-08-01,0.18035,0.78740,0.530580,-1.63324,-2.1,3.0,-0.052340,-0.008281,0.420013,-0.034826,-0.024273,-0.008281,0.420013
204,2023-09-01,0.16686,4.10156,0.511447,-3.31905,-0.7,2.4,-0.030673,-0.047472,0.073404,-0.009513,-0.191162,-0.047472,0.073404
205,2023-10-01,0.63119,0.00000,0.511447,-1.04501,-0.1,5.9,-1.404270,-0.086984,-0.086425,-0.025103,0.135212,-0.086984,-0.086425


In [7]:
outputs = pd.read_csv('../Data/all_outputs_cleaned.csv')
outputs.drop(['Unnamed: 0'], axis = 1, inplace = True)
outputs

,Month,Construction,Information,Total_Private
0,2006-05-01,-0.001683,-0.002626,0.000254
1,2006-06-01,-0.001815,-0.000987,0.000744
2,2006-07-01,0.001689,-0.001647,0.001250
3,2006-08-01,0.001037,0.001320,0.001117
4,2006-09-01,-0.000259,-0.002306,0.000567
...,...,...,...,...
200,2023-05-01,0.003164,-0.001292,0.001916
201,2023-06-01,0.003659,-0.003235,0.000645
202,2023-07-01,0.001508,-0.006167,0.001087
203,2023-08-01,0.004519,-0.006858,0.001325


# Recurrent Neural Network